In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import pickle


from src.visualization import (
    visualize_network_topology,
    visualize_network_with_failures,
    visualize_grid_improvement
)

from src.functions import (
    build_and_simplify_network,
    find_failures_with_yearly_profile,
    suggest_grid_reinforcement,
    print_analysis_results,
)


# --- Start: Loading and Preparing the Data ---
data_path = "../challenge-data/edih-data/"


file_path = data_path + "250827_all_stations_anon_correct.csv"
df_full = pd.read_csv(file_path)
print("Successfully loaded network data.")


profiles_path = data_path + "data_parquet/"
df_consumption = pd.read_parquet(profiles_path + "base_consumption.parquet").set_index('timestamp')
df_pv = pd.read_parquet(profiles_path + "pv_profiles.parquet").set_index('timestamp')
df_ev = pd.read_parquet(profiles_path + "ev_profiles.parquet").set_index('timestamp')
df_hp = pd.read_parquet(profiles_path + "hp_profiles.parquet").set_index('timestamp')
print("Successfully loaded and indexed all profiles data.")


df_reinforcement = pd.read_csv(data_path + "190923_Einheitskosten_Invest.csv")
print("Successfully loaded grid reinforcement costs.")

Successfully loaded network data.
Successfully loaded and indexed all profiles data.
Successfully loaded grid reinforcement costs.


In [2]:
NOMINAL_VOLTAGE = 400.0
#df_net_load = df_consumption.add(df_ev, fill_value=0).add(df_hp, fill_value=0).subtract(df_pv, fill_value=0)
df_net_load = df_consumption.copy()
print("Successfully calculated net load profile.")

Successfully calculated net load profile.


In [3]:
# ==============================================================================
# --- JUPYTER-FRIENDLY INTERACTIVE WIDGETS ---
# ==============================================================================
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Get a list of all unique stations
all_stations = sorted(df_full['station'].unique())

# 1. Create the dropdown widget
station_dropdown = widgets.Dropdown(
    options=all_stations,
    description='Select Station:',
    value=all_stations[0] if all_stations else None,
    disabled=False,
)

# 2. Create an output widget to hold the prints and plots
output_area = widgets.Output()

# 3. Define the function that runs when the dropdown value changes
def on_station_change(change):
    selected_station = change['new']
    
    with output_area:
        clear_output(wait=True)
        print(f"--- Running full analysis for station: '{selected_station}' ---")
        
        # --- STEP 1: Build the network for the selected station ---
        df_one_station = df_full[df_full['station'] == selected_station].copy()
        G, consumer_props, roots = build_and_simplify_network(df_one_station)
        
        print("\n--- Basic Network Info ---")
        print(f"Simplified graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
        print(f"Identified {len(roots)} root node(s): {roots}")
        print(f"Found properties for {len(consumer_props)} unique consumers.")
        
        print("\n--- 1. Initial Network Topology ---")
        visualize_network_topology(
            graph=G, 
            root_node_ids=roots, 
            optimize_space=True,
            station_name=selected_station
        )
        
        # --- STEP 2: Run dynamic profile-based analysis to find failures ---
        print("\n" + "="*50)
        print("Running yearly profile analysis to check for failures...")
        dynamic_results = find_failures_with_yearly_profile(
            graph=G,
            net_profile_df=df_net_load,
            consumer_props=consumer_props,
            root_node_ids=roots,
            nominal_voltage=NOMINAL_VOLTAGE
        )
        print_analysis_results("2. RESULTS: Dynamic Yearly Profile Analysis", dynamic_results)
        
        # --- STEP 3: Suggest grid reinforcement if needed ---
        print("\n" + "="*50)
        print("Calculating suggested grid reinforcements...")
        reinforcement_results = suggest_grid_reinforcement(
            initial_graph=G,
            initial_results=dynamic_results,
            reinforcement_costs_df=df_reinforcement,
            net_profile_df=df_net_load,
            consumer_props=consumer_props,
            root_node_ids=roots,
            nominal_voltage=NOMINAL_VOLTAGE
        )
        
        # Display the final plan and cost
        print("\n" + "#"*40 + "\n### 3. FINAL REINFORCEMENT SUMMARY ###\n" + "#"*40)
        print(f"\nFinal Status: {reinforcement_results['status']}")
        print(f"Total Estimated Cost: {reinforcement_results['total_cost_CHF']:.2f} CHF")
        
        if 'reinforcement_plan' in reinforcement_results and not reinforcement_results['reinforcement_plan'].empty:
            print("\nRecommended Reinforcement Actions:")
            display(reinforcement_results['reinforcement_plan'])
        else:
            print("\nNo reinforcement actions were needed or suggested.")
            
        # --- STEP 4: Visualize the grid improvement (Before vs. After) ---
        # Only show this visualization if the reinforcement was successful and there was a plan
        if (reinforcement_results['status'] == 'Success' and 
            'reinforcement_plan' in reinforcement_results and 
            not reinforcement_results['reinforcement_plan'].empty):
            
            print("\n" + "#"*40 + "\n### 4. VISUALIZATION: Grid Improvement ###\n" + "#"*40)
            visualize_grid_improvement(
                initial_graph=G,
                initial_failures=dynamic_results['link_failures'],
                reinforced_graph=reinforcement_results['reinforced_graph'],
                reinforcement_plan=reinforcement_results['reinforcement_plan'],
                root_node_ids=roots,
                total_cost=reinforcement_results['total_cost_CHF'],
                optimize_space=True,
                station_name=selected_station
            )
        else:
            print("\n" + "#"*40 + "\n### 4. VISUALIZATION: Grid Failures ###\n" + "#"*40)
            visualize_network_with_failures(
                graph=G,
                root_node_id=roots,
                link_failures=dynamic_results["link_failures"],
                fuse_failures=dynamic_results["fuse_failures"],
                optimize_space=True,
                station_name=selected_station
            )

# 4. Link the function to the dropdown widget
station_dropdown.observe(on_station_change, names='value')

# 5. Display the UI and run the analysis for the initial default value
print("Use the dropdown below to select a station for a full grid analysis.")
display(station_dropdown, output_area)

if station_dropdown.value:
    on_station_change({'new': station_dropdown.value})

Use the dropdown below to select a station for a full grid analysis.


Dropdown(description='Select Station:', options=('station_1', 'station_2', 'station_3', 'station_4', 'station_…

Output()